# Running a multi-member hydrological ensemble on the Raven Server

Here we use birdy's WPS client to launch the GR4JCN and HMETS hydrological models on the server and analyze the outputs. 

In [1]:
from birdy import WPSClient

from example_data import TESTDATA
import datetime as dt
from urllib.request import urlretrieve
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt

url = "http://localhost:9099/wps"
wps = WPSClient(url)

In [2]:
# The model parameters for gr4jcn and hmets. Can either be a string of comma separated values, a list, an array or a named tuple. 
gr4jcn1 ='0.529, -3.396, 407.29, 1.072, 16.9, 0.947'
gr4jcn2 ='0.4, -3.96, 307.29, 1.072, 16.9, 0.947'

params = [gr4jcn1,gr4jcn2] # Rich Test 1



# Forcing files. Raven uses the same forcing files for all and extracts the information it requires for each model.
ts=TESTDATA['raven-gr4j-cemaneige-nc-ts']

# Model configuration parameters.
config = dict(
    start_date=dt.datetime(2000, 1, 1),
    end_date=dt.datetime(2002, 1, 1),
    area=4250.6,
    elevation=843.0,
    latitude=54.4848,
    longitude=-123.3659,
    )

# Launch the WPS to get the multi-model results.  Note the "gr4jcn" and "hmets" keys.
resp=wps.raven_gr4j_cemaneige(ts=str(ts),params=params, **config) 




In [3]:
# And get the response
# With `asobj` set to False, only the reference to the output is returned in the response. 
# Setting `asobj` to True does nt work here as the outputs are zipped files and could be numerous.
# The user can download the resulting data. So we use asobj=False
[hydrograph, storage, solution, diagnostics] = resp.get(asobj=True)


/home/david/src/birdy/birdy/client/converters.py:294: UserWarning: No converter was found for /tmp/tmpx88te0jk/raven-gr4j-cemaneige-sim-0_solution.rvc
  warnings.warn(UserWarning("No converter was found for {}".format(output)))
/home/david/src/birdy/birdy/client/converters.py:294: UserWarning: No converter was found for /tmp/tmpx88te0jk/raven-gr4j-cemaneige-sim-1_solution.rvc
  warnings.warn(UserWarning("No converter was found for {}".format(output)))


In [4]:
print(hydrograph)
print(diagnostics)

<xarray.Dataset>
Dimensions:     (nbasins: 1, params: 2, time: 732)
Coordinates:
  * time        (time) datetime64[ns] 2000-01-01 2000-01-02 ... 2002-01-01
    basin_name  (nbasins) object ...
Dimensions without coordinates: nbasins, params
Data variables:
    precip      (time) float64 ...
    q_obs       (time, nbasins) float64 ...
    q_in        (time, nbasins) float64 ...
    q_sim       (params, time, nbasins) float64 ...
Attributes:
    Conventions:          CF-1.6
    featureType:          timeSeries
    history:              Created on 2019-06-21 14:00:20 by Raven
    description:          Standard Output
    title:                Simulated river discharge
    references:           Craig J.R. and the Raven Development Team Raven use...
    comment:              Raven Hydrological Framework version 2.9 rev#177
    model_id:             gr4jcn
    time_frequency:       day
    time_coverage_start:  2000-01-01 00:00:00
    time_coverage_end:    2002-01-01 00:00:00
['observed data

The `hydrograph` and `diagnostics` files can be downloaded by clicking the above links. hydrographs is a netcdf file containing the multiple simulations, and diagnostics is a zip-file containing the various diagnostics files linked to each parameter set.